In [92]:
# Menyiapkan library
import seaborn as sns
sns.set()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [93]:
# Load dataset
df = pd.read_csv('unburn_carbon3.csv')

# Melihat tampilan dataset
df.head()

,time,unburn_carbon,nilai_kalor,co,co2,out_generator,sfc,eco_gas_out_temp,pa_flow,o2,...,sa_flow,carbon,hydrogen,nitrogen,sulfur,ash,total_moisture,oxygen,surface_moisture,inherent_moisture
0,13-Apr-16,3.6590,4559.685911,0.0254,16.850,609.131028,0.515488,353.347333,928.582025,1.2775,...,1422.268295,49.372898,3.710498,0.588508,0.498562,7.304680,28.997219,9.527635,11.598888,17.398331
1,09-May-16,1.5045,4739.941890,0.0000,13.250,620.713631,0.507914,375.683444,1016.945563,4.0975,...,1408.914527,48.169496,3.481469,0.760060,0.432251,7.895477,27.330060,11.931188,10.932024,16.398036
2,07-Jun-16,2.4490,4832.126248,0.0000,13.275,607.201920,0.512293,353.364613,966.932480,3.3000,...,1363.565909,50.586591,3.739764,0.700430,0.432364,8.154385,26.550000,9.836467,10.620000,15.930000
3,22-Jul-16,0.9680,4599.993082,0.0757,13.275,632.613046,0.543439,356.020724,948.621111,4.3350,...,1471.384697,48.348807,3.309546,0.671983,0.540600,7.817141,28.855547,10.456376,11.542219,17.313328
4,09-Aug-16,3.9520,5067.000000,0.1556,18.500,602.272434,0.516813,357.003215,941.040629,0.6075,...,1480.650494,51.370000,3.730000,0.720000,0.640000,4.670000,26.540000,12.330000,10.616000,15.924000


In [94]:

df.drop('time', axis=1, inplace=True)

In [95]:
df = df[(df['unburn_carbon'] < 2)]

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 1 to 81
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   unburn_carbon      74 non-null     float64
 1   nilai_kalor        74 non-null     float64
 2   co                 74 non-null     float64
 3   co2                74 non-null     float64
 4   out_generator      72 non-null     float64
 5   sfc                72 non-null     float64
 6   eco_gas_out_temp   73 non-null     float64
 7   pa_flow            74 non-null     float64
 8   o2                 74 non-null     float64
 9   fuel_temp          74 non-null     float64
 10  sa_flow            74 non-null     float64
 11  carbon             74 non-null     float64
 12  hydrogen           74 non-null     float64
 13  nitrogen           74 non-null     float64
 14  sulfur             74 non-null     float64
 15  ash                74 non-null     float64
 16  total_moisture     74 non-nu

In [97]:
df.isnull().sum()

unburn_carbon        0
nilai_kalor          0
co                   0
co2                  0
out_generator        2
sfc                  2
eco_gas_out_temp     1
pa_flow              0
o2                   0
fuel_temp            0
sa_flow              0
carbon               0
hydrogen             0
nitrogen             0
sulfur               0
ash                  0
total_moisture       0
oxygen               0
surface_moisture     0
inherent_moisture    0
dtype: int64

In [98]:
for kolom in df.columns:
    median_kolom = df[kolom].median()
    df[kolom].fillna(median_kolom, inplace=True)

In [99]:
df.isnull().sum()

unburn_carbon        0
nilai_kalor          0
co                   0
co2                  0
out_generator        0
sfc                  0
eco_gas_out_temp     0
pa_flow              0
o2                   0
fuel_temp            0
sa_flow              0
carbon               0
hydrogen             0
nitrogen             0
sulfur               0
ash                  0
total_moisture       0
oxygen               0
surface_moisture     0
inherent_moisture    0
dtype: int64

In [100]:
df.head()

,unburn_carbon,nilai_kalor,co,co2,out_generator,sfc,eco_gas_out_temp,pa_flow,o2,fuel_temp,sa_flow,carbon,hydrogen,nitrogen,sulfur,ash,total_moisture,oxygen,surface_moisture,inherent_moisture
1,1.5045,4739.941890,0.00000,13.250,620.713631,0.507914,375.683444,1016.945563,4.0975,60.332083,1408.914527,48.169496,3.481469,0.760060,0.432251,7.895477,27.330060,11.931188,10.932024,16.398036
3,0.9680,4599.993082,0.07570,13.275,632.613046,0.543439,356.020724,948.621111,4.3350,60.332083,1471.384697,48.348807,3.309546,0.671983,0.540600,7.817141,28.855547,10.456376,11.542219,17.313328
5,1.6795,4398.000000,0.00015,15.625,501.593013,0.612977,361.808183,897.331199,3.3575,60.332083,1471.429447,50.770000,3.240000,0.570000,0.140000,2.940000,32.300000,10.040000,12.920000,19.380000
6,0.3195,3375.902050,0.00015,12.700,312.332833,0.647850,360.419045,730.391084,6.7300,60.332083,1179.142903,35.864958,4.043400,0.498686,0.276299,5.950537,32.610000,20.756120,13.044000,19.566000
7,0.1360,3677.494560,0.28690,17.650,487.335794,0.569324,335.432489,873.055673,1.4500,60.332083,1226.166930,39.115592,4.293008,0.615240,0.314456,4.450236,31.640000,19.571468,12.656000,18.984000


In [101]:
df = pd.DataFrame(df)

# Inisialisasi objek MinMaxScaler
scaler = MinMaxScaler()

# Menghitung min dan max, serta melakukan scaling
df_scaled = scaler.fit_transform(df)

# Mengonversi ndarray menjadi DataFrame
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
print(df_scaled)

    unburn_carbon  nilai_kalor        co       co2  out_generator       sfc  \
0        0.894102     0.865437  0.000000  0.644013       0.962847  0.079420   
1        0.569451     0.776644  0.263855  0.646036       1.000000  0.187603   
2        1.000000     0.648487  0.000523  0.836165       0.590921  0.399357   
3        0.177025     0.000000  0.000523  0.599515       0.000000  0.505549   
4        0.065984     0.191350  1.000000  1.000000       0.546406  0.266427   
..            ...          ...       ...       ...            ...       ...   
69       0.014548     1.000000  0.001182  0.498382       0.824068  0.406396   
70       0.075666     0.979842  0.000630  0.771440       0.490031  0.000000   
71       0.071127     0.386106  0.045695  0.882686       0.818743  0.315714   
72       0.183984     0.277349  0.045695  0.882686       0.806504  0.464193   
73       0.044804     0.251181  0.001318  0.956311       0.771923  0.465846   

    eco_gas_out_temp   pa_flow        o2  fuel_temp

In [102]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['unburn_carbon'], axis=1),
    df['unburn_carbon'],
    test_size = 0.2,
    random_state = 0)

# Melihat dimensi training set dan test set
print((X_train.shape), (X_test.shape))

(59, 19) (15, 19)


In [103]:
import lightgbm as lgb

# Inisialisasi model LightGBM Regressor
model = lgb.LGBMRegressor(random_state=0)

# Melatih model pada data pelatihan
model.fit(X_train, y_train)

# Memprediksi target pada data pengujian
y_pred = model.predict(X_test)

# Mengukur kinerja model dengan Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 0.2731182784004


In [104]:
import pickle

# Menyimpan model
pickle.dump(model, open('unburn_model2.pkl', 'wb'))